In [1]:
# Load SQL extension
%load_ext sql

In [2]:
%%sql sqlite:///movies_data.db

-- Example SQL query
SELECT COUNT(*) FROM actors;

Done.


COUNT(*)
145


### Number of customer by countried and genders

In [ ]:
%%sql
-- Count the total number of customers, the number of customers for each country, and the number of female and male customers for each country
SELECT country,
       gender,
	   COUNT(*)
FROM customers
GROUP BY ROLLUP (country, gender)
ORDER BY country, gender; -- Order the result by country and gender

#### Group of customers
Use the CUBE operator to extract the content of a pivot table from the database. Create a table with the total number of male and female customers from each country.

In [ ]:
%%sql
SELECT gender, -- Extract information of a pivot table of gender and country for the number of customers
	   country,
	   COUNT(*)
FROM customers
GROUP BY CUBE (country, gender)
ORDER BY country;

#### Categories of movies
Give an overview on the movies available on MovieNow. List the number of movies for different genres and release years.

In [ ]:
SELECT genre,
       year_of_release,
       COUNT(*)
FROM movies
GROUP BY CUBE (genre,year_of_release)
ORDER BY year_of_release;

### Customer Preference for Genre

In [4]:
%%sql
SELECT m.genre, -- For each genre, calculate:
       AVG(r.rating) AS avg_rating, -- The average rating and use the alias avg_rating
       COUNT(r.rating) AS n_rating, -- The number of ratings and use the alias n_rating
       COUNT(*) AS n_rentals,     -- The number of movie rentals and use the alias n_rentals
       COUNT(DISTINCT r.movie_id) as n_movies -- The number of distinct movies and use the alias n_movies
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( 
    SELECT movie_id
    FROM renting
    GROUP BY movie_id
    HAVING COUNT(rating) >= 3 )
AND r.date_renting >= '2018-01-01'
GROUP BY genre
ORDER BY avg_rating DESC ; -- Order the table by decreasing average rating

 * sqlite:///movies_data.db
Done.


genre,avg_rating,n_rating,n_rentals,n_movies
Action & Adventure,8.714285714285714,7,9,2
Art House & International,8.5,4,5,1
Other,8.428571428571429,7,16,2
Science Fiction & Fantasy,8.27659574468085,47,70,10
Comedy,7.95,20,31,5
Animation,7.833333333333333,6,10,2
Drama,7.748251748251748,143,245,34
Mystery & Suspense,7.428571428571429,7,19,3


### Customer preference for actors

In [10]:
%%sql
-- Group by each county and genre with OLAP extension
SELECT 
	c.country, 
	m.genre, 
	AVG(r.rating) AS avg_rating, 
	COUNT(*) AS num_rating
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY ROLLUP (country, genre)
ORDER BY c.country, m.genre;

 * sqlite:///movies_data.db
(sqlite3.OperationalError) no such function: ROLLUP
[SQL: -- Group by each county and genre with OLAP extension
SELECT 
	c.country, 
	m.genre, 
	AVG(r.rating) AS avg_rating, 
	COUNT(*) AS num_rating
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY ROLLUP (country, genre)
ORDER BY c.country, m.genre;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [9]:
%%sql
SELECT a.nationality,
       a.gender,
       AVG(r.rating) AS avg_rating, -- The average rating
       COUNT(r.rating) AS n_rating, -- The number of ratings
       COUNT(*) AS n_rentals, -- The number of movie rentals
       COUNT(DISTINCT a.actor_id) AS n_actors -- The number of actors
FROM renting AS r
LEFT JOIN actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN actors AS a
ON ai.actor_id = a.actor_id
WHERE r.movie_id IN ( 
    SELECT movie_id
    FROM renting
    GROUP BY movie_ids
    HAVING COUNT(rating) >=4 )
AND r.date_renting >= '2018-04-01'
GROUP BY CUBE (a.nationality,a.gender); -- Report results for each combination of the actors' nationality and gender WITH CUBE

 * sqlite:///movies_data.db
(sqlite3.OperationalError) no such function: ROLLUP
[SQL: SELECT a.nationality,
       a.gender,
	   AVG(r.rating) AS avg_rating, -- The average rating
	   COUNT(r.rating) AS n_rating, -- The number of ratings
	   COUNT(*) AS n_rentals, -- The number of movie rentals
	   COUNT(DISTINCT a.actor_id) AS n_actors -- The number of actors
FROM renting AS r
LEFT JOIN actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN actors AS a
ON ai.actor_id = a.actor_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(rating) >=4 )
AND r.date_renting >= '2018-04-01'
GROUP BY ROLLUP (a.nationality,a.gender); -- Report results for each combination of the actors' nationality and gender WITH CUBE]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
